In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
data = pd.read_csv('./data/Churn_Modelling.csv')

In [3]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
data.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [5]:
# First three columns are not required
# Preprocessing

data.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1, inplace=True)

In [6]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
data.describe

<bound method NDFrame.describe of       CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0             619    France  Female   42       2       0.00              1   
1             608     Spain  Female   41       1   83807.86              1   
2             502    France  Female   42       8  159660.80              3   
3             699    France  Female   39       1       0.00              2   
4             850     Spain  Female   43       2  125510.82              1   
...           ...       ...     ...  ...     ...        ...            ...   
9995          771    France    Male   39       5       0.00              2   
9996          516    France    Male   35      10   57369.61              1   
9997          709    France  Female   36       7       0.00              1   
9998          772   Germany    Male   42       3   75075.31              2   
9999          792    France  Female   28       4  130142.79              1   

      HasCrCard  IsActiveMemb

In [8]:
data.dtypes

CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [9]:
## Encoding categorical variables

label_encoder_gender = LabelEncoder()

data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [10]:
## One hot encoder for Geography

from sklearn.preprocessing import OneHotEncoder

In [11]:
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [12]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [13]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], shape=(10000, 3))

In [14]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [15]:
## Combinme

data = pd.concat([data.drop('Geography', axis = 1), geo_encoded_df], axis = 1)

In [16]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [17]:
## Save the encoders and scaler

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

In [18]:
# Diving the data in independent and dependent features

X = data.drop('Exited', axis = 1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# Scale the features

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [20]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]], shape=(8000, 12))

In [21]:
#ANN Implementation

In [22]:
import tensorflow as tf


1. Sequential NW
2. Dense Neurons
3. Activation Fn 
4. Optimzie -> Backpropogation -> Updating the weights
5. Loss function -> need to reduce
6. Metrics -> Accuracy, [mse, mae]
7. Training -> Logs -> Folder > Tensorboard


In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [24]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1], )),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

C:\ProgramData\anaconda3\envs\tf-nn\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01 )
model.compile(optimizer=opt, loss='binary_crossentropy',metrics=['accuracy'])

In [27]:
##setup tensorboard

log_dir = 'logs/fit' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [28]:
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [29]:
## Setup Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

In [30]:
### Training model 

history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs = 100,
    callbacks=[tensorflow_callback, early_stopping]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8304 - loss: 0.4067 - val_accuracy: 0.8590 - val_loss: 0.3483
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 940us/step - accuracy: 0.8564 - loss: 0.3515 - val_accuracy: 0.8630 - val_loss: 0.3518
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 946us/step - accuracy: 0.8591 - loss: 0.3469 - val_accuracy: 0.8585 - val_loss: 0.3492
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 920us/step - accuracy: 0.8597 - loss: 0.3423 - val_accuracy: 0.8620 - val_loss: 0.3459
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 915us/step - accuracy: 0.8634 - loss: 0.3384 - val_accuracy: 0.8565 - val_loss: 0.3434
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 927us/step - accuracy: 0.8636 - loss: 0.3378 - val_accuracy: 0.8630 - val_loss: 0.3474
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 967us/step - accuracy: 0.8641 - loss: 0.3355 - val_accuracy: 0.8625 - val_loss: 0.3395
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 992us/step - accuracy: 0.8622 - loss: 0.3

250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8585 - loss: 0.3496 - val_accuracy: 0.8555 - val_loss: 0.3511
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 894us/step - accuracy: 0.8589 - loss: 0.3442 - val_accuracy: 0.8560 - val_loss: 0.3494
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 882us/step - accuracy: 0.8608 - loss: 0.3383 - val_accuracy: 0.8625 - val_loss: 0.3412
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 880us/step - accuracy: 0.8637 - loss: 0.3331 - val_accuracy: 0.8605 - val_loss: 0.3440
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 921us/step - accuracy: 0.8609 - loss: 0.3378 - val_accuracy: 0.8560 - val_loss: 0.3434
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 895us/step - accuracy: 0.8669 - loss: 0.3332 - val_accuracy: 0.8565 - val_loss: 0.3389
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 908us/step - accuracy: 0.8626 - loss: 0.3325 - val_accuracy: 0.8560 - val_loss: 0.3399
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 931us/step - accuracy: 0.8659 - loss: 0.3324 - val_ac

In [31]:
model.save('model.h5')

In [32]:
## Load Tensorboard extension

%load_ext tensorboard

In [35]:
%tensorboard --logdir logs/fit20260205-120647